In [1]:
import os
import shutil
from pathlib import Path
import numpy as np
import random

np.random.seed(0) 
random.seed(0)


In [2]:
import os
import shutil
from pathlib import Path
import numpy as np
import random
import torch
import optuna
from ultralytics import YOLO
import yaml

In [3]:
class AdvancedYOLOTrainer:
    def __init__(self, base_model="yolo11n.pt"):
        self.base_model = base_model
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
    def create_advanced_config(self):
        config = {
            # Hiperparâmetros otimizados para contexto cirúrgico
            'lr0': 0.001,  # Fine-tuning em datasets cirúrgicos
            'lrf': 0.01,   # Agendamento coseno com warmup
            'momentum': 0.98,  # Otimizado para dados com artefatos
            'weight_decay': 0.0005,
            'warmup_epochs': 10,  # Previne overfitting inicial
            'warmup_momentum': 0.8,
            'warmup_bias_lr': 0.1,
            
            # Configurações de loss otimizadas
            'box': 7.5,
            'cls': 0.5,
            'dfl': 1.5,
            
            # Configurações de treino
            'epochs': 150,
            'batch': 8,
            'imgsz': 640,
            'patience': 50,
            'save_period': 10,
            
            # Augmentações específicas para ambiente cirúrgico
            'hsv_h': 0.015,  # Pequenos ajustes de matiz
            'hsv_s': 0.7,    # Saturação para instrumentos metálicos
            'hsv_v': 0.4,    # Valor para compensar subexposição
            'degrees': 15.0,  # Rotação limitada
            'translate': 0.1,
            'scale': 0.5,
            'shear': 5.0,
            'perspective': 0.0001,
            'flipud': 0.0,   # Sem flip vertical em cirurgia
            'fliplr': 0.5,   # Flip horizontal permitido
            'mosaic': 1.0,
            'mixup': 0.15,   # Mixup reduzido para preservar contexto
            'copy_paste': 0.3,  # Copy-paste para instrumentos
            
            # Regularização avançada
            'dropout': 0.2,  # Stochastic depth
            'val': True,
            'plots': True,
            'verbose': True
        }
        return config

    def setup_multiphase_training(self):
        phases = {
            'phase1': {
                'description': 'COCO pre-training com backbone congelado',
                'epochs': 50,
                'freeze': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],  # Congela backbone
                'lr0': 0.01,
                'momentum': 0.937
            },
            'phase2': {
                'description': 'Fine-tuning completo em dados cirúrgicos',
                'epochs': 100,
                'freeze': None,
                'lr0': 0.001,
                'momentum': 0.98
            },
            'phase3': {
                'description': 'Adaptação final com regularização forte',
                'epochs': 50,
                'freeze': None,
                'lr0': 0.0001,
                'momentum': 0.99,
                'weight_decay': 0.001
            }
        }
        return phases

    def create_custom_model_config(self):
        model_config = {
            # Backbone otimizado
            'backbone': {
                'type': 'YOLOv11Backbone',
                'depth_multiple': 0.33,
                'width_multiple': 0.25,
                'max_channels': 1024
            },
            
            # Neck com feature pyramid melhorado
            'neck': {
                'type': 'YOLOv11Neck',
                'in_channels': [256, 512, 1024],
                'out_channels': 256,
                'num_outs': 3
            },
            
            # Head especializado para instrumentos alongados
            'head': {
                'type': 'YOLOv11Head',
                'num_classes': 4,  # needle, clamp, curved_scissor, straight_scissor
                'in_channels': 256,
                'feat_channels': 256,
                'anchor_generator': {
                    'type': 'YOLOAnchorGenerator',
                    'base_sizes': [[(10, 13), (16, 30), (33, 23)],
                                   [(30, 61), (62, 45), (59, 119)],
                                   [(116, 90), (156, 198), (373, 326)]],
                    'strides': [8, 16, 32]
                }
            }
        }
        return model_config

    def optimize_hyperparameters(self, n_trials=50):
        def objective(trial):
            # Parâmetros a otimizar
            lr0 = trial.suggest_float('lr0', 1e-5, 1e-1, log=True)
            momentum = trial.suggest_float('momentum', 0.8, 0.99)
            weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-2, log=True)
            box_loss = trial.suggest_float('box', 5.0, 10.0)
            cls_loss = trial.suggest_float('cls', 0.3, 1.0)
            
            # Treino com parâmetros sugeridos
            model = YOLO(self.base_model)
            results = model.train(
                data=Path("yolo_dataset/data.yaml").resolve(),
                epochs=10,  
                lr0=lr0,
                momentum=momentum,
                weight_decay=weight_decay,
                box=box_loss,
                cls=cls_loss,
                verbose=False,
                plots=False
            )
            
            # Retorna mAP@0.5 como métrica objetivo
            return results.results_dict.get('metrics/mAP50(B)', 0.0)
        
        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=n_trials)
        
        return study.best_params

    def train_with_advanced_features(self, data_path, use_optimization=True):
        
        if use_optimization:
            best_params = self.optimize_hyperparameters(n_trials=20)
        else:
            best_params = {}
        
        # Configuração base
        config = self.create_advanced_config()
        config.update(best_params)  # Aplica parâmetros otimizados
        
        # Fases de treino
        phases = self.setup_multiphase_training()
        
        best_model = None
        best_map = 0.0
        
        for phase_name, phase_config in phases.items():
            
            # Carrega modelo
            if phase_name == 'phase1':
                model = YOLO(self.base_model)
            else:
                model = YOLO('runs/detect/train/weights/best.pt')
            
            # Atualiza configuração para a fase
            current_config = config.copy()
            current_config.update({
                'epochs': phase_config['epochs'],
                'lr0': phase_config.get('lr0', config['lr0']),
                'momentum': phase_config.get('momentum', config['momentum']),
                'freeze': phase_config.get('freeze'),
                'name': f'train_{phase_name}'
            })
            
            # Treino da fase
            results = model.train(
                data=data_path,
                **current_config
            )
            
            # Avalia resultados
            current_map = results.results_dict.get('metrics/mAP50(B)', 0.0)
            if current_map > best_map:
                best_map = current_map
                best_model = f'runs/detect/train_{phase_name}/weights/best.pt'
            
        return best_model, best_map

    def apply_advanced_augmentations(self):
        augmentation_config = {
            # Transformações geométricas
            'degrees': 15.0,  # Rotação limitada
            'translate': 0.1,
            'scale': 0.5,
            'shear': 5.0,
            'perspective': 0.0001,
            
            # Transformações fotométricas otimizadas
            'hsv_h': 0.015,  # Matiz para luzes cirúrgicas
            'hsv_s': 0.7,    # Saturação para metais
            'hsv_v': 0.4,    # Compensação de subexposição
            
            # Augmentações específicas
            'mosaic': 1.0,
            'mixup': 0.15,
            'copy_paste': 0.3,
            'erasing': 0.4,  # Simula oclusões
            
            # Sem flips verticais (não realista em cirurgia)
            'flipud': 0.0,
            'fliplr': 0.5
        }
        return augmentation_config

In [4]:
def main():
    
    trainer = AdvancedYOLOTrainer("yolo11n.pt")
    
    root = Path("../Surgical-Dataset")
    img_dir = root / "Images"
    obj_label_dir = root / "Labels/label object names"
    
    yolo_dir = Path("yolo_dataset")
    (yolo_dir / "images/train").mkdir(parents=True, exist_ok=True)
    (yolo_dir / "labels/train").mkdir(parents=True, exist_ok=True)
    
    for label_file in obj_label_dir.glob("*.txt"):
        img_id = label_file.stem
        img_file = img_dir / f"{img_id}.jpg"
        if img_file.exists():
            shutil.copy(img_file, yolo_dir / "images/train" / f"{img_id}.jpg")
            shutil.copy(label_file, yolo_dir / "labels/train" / f"{img_id}.txt")
    
    label_map = {
        0: "bisturi",
        1: "pinca", 
        2: "tesouracurva",
        3: "tesourareta",
    }
    
    label_remap = {
        "bisturi": "needle",
        "pinca": "clamp", 
        "tesouracurva": "curved_scissor",
        "tesourareta": "straight_scissor"
    }
    
    with open(yolo_dir / "classes.txt", "w") as f:
        for _, name in sorted(label_map.items()):
            f.write(f"{label_remap.get(name, name)}\n")
    
    data_config = {
        'path': str(yolo_dir.absolute()),
        'train': 'images/train',
        'val': 'images/train',  # Usando mesmo conjunto para demonstração
        'nc': 4,
        'names': [label_remap.get(label_map[i], label_map[i]) for i in sorted(label_map.keys())]
    }
    
    with open(yolo_dir / "data.yaml", "w") as f:
        yaml.dump(data_config, f)
    
    # **TREINO AVANÇADO COM TODAS AS MELHORIAS**
    print("A iniciar treino avançado com melhorias do relatório...")
    
    data_path = Path("yolo_dataset/data.yaml").resolve()
    best_model_path, best_map = trainer.train_with_advanced_features(
        data_path=data_path,
        use_optimization=True  # Ativa otimização de hiperparâmetros
    )
    
    print(f"Treino concluído!")
    print(f"Melhor mAP@0.5: {best_map:.4f}")
    print(f"Melhor modelo salvo em: {best_model_path}")
    
    # Validação final com métricas avançadas
    if best_model_path:
        final_model = YOLO(best_model_path)
        
        # Teste de inferência
        results = final_model.val(data=data_path)
        
        # Métricas detalhadas por classe
        print("\nMétricas por classe:")
        for i, class_name in enumerate(data_config['names']):
            if hasattr(results, 'maps') and len(results.maps) > i:
                print(f"  {class_name}: mAP@0.5 = {results.maps[i]:.4f}")
        
        print(f"\nmAP@0.5 geral: {results.box.map50:.4f}")
        print(f"mAP@0.5:0.95 geral: {results.box.map:.4f}")
        print(f"Velocidade de inferência: {results.speed['inference']:.1f}ms")

if __name__ == "__main__":
    main()


[I 2025-05-25 18:39:14,969] A new study created in memory with name: no-name-b3ce9c51-761a-40e1-8e69-7d98b7d47413


A iniciar treino avançado com melhorias do relatório...
New https://pypi.org/project/ultralytics/8.3.144 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.139 🚀 Python-3.10.16 torch-2.7.0+cu128 CUDA:0 (NVIDIA RTX A6000, 48548MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=8.32067095574515, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.7969444412090911, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/mounts/grupo1/yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0002060533367160293, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, mod

train: Scanning /mounts/grupo1/yolo_dataset/labels/train.cache... 3009 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3009/3009 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1759.6±789.9 MB/s, size: 77.6 KB)


val: Scanning /mounts/grupo1/yolo_dataset/labels/train.cache... 3009 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3009/3009 [00:00<?, ?it/s]


...


50 epochs completed in 0.705 hours.
Optimizer stripped from runs/detect/train_phase3/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train_phase3/weights/best.pt, 5.5MB

Validating runs/detect/train_phase3/weights/best.pt...
Ultralytics 8.3.139 🚀 Python-3.10.16 torch-2.7.0+cu128 CUDA:0 (NVIDIA RTX A6000, 48548MiB)
YOLO11n summary (fused): 100 layers, 2,582,932 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 189/189 [00:10<00:00, 17.73it/s]


                   all       3009       4207      0.881      0.884      0.946      0.737
                needle       1038       1038       0.92      0.847      0.949      0.709
                 clamp       1056       1056      0.909      0.846       0.93      0.671
        curved_scissor        971        971      0.828      0.916      0.948       0.78
      straight_scissor       1142       1142      0.866      0.926      0.958      0.789
Speed: 0.1ms preprocess, 0.5ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/train_phase3
Treino concluído!
Melhor mAP@0.5: 0.9786
Melhor modelo salvo em: runs/detect/train_phase2/weights/best.pt
Ultralytics 8.3.139 🚀 Python-3.10.16 torch-2.7.0+cu128 CUDA:0 (NVIDIA RTX A6000, 48548MiB)
YOLO11n summary (fused): 100 layers, 2,582,932 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1847.9±777.6 MB/s, size: 71.1 KB)


val: Scanning /mounts/grupo1/yolo_dataset/labels/train.cache... 3009 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3009/3009 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 189/189 [00:12<00:00, 14.56it/s]


                   all       3009       4207      0.942      0.939      0.979      0.777
                needle       1038       1038      0.965      0.915      0.979      0.751
                 clamp       1056       1056       0.93      0.912      0.964      0.704
        curved_scissor        971        971      0.927      0.958      0.984      0.828
      straight_scissor       1142       1142      0.945      0.969      0.989      0.827
Speed: 0.2ms preprocess, 0.9ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/val2

Métricas por classe:
  needle: mAP@0.5 = 0.7507
  clamp: mAP@0.5 = 0.7036
  curved_scissor: mAP@0.5 = 0.8275
  straight_scissor: mAP@0.5 = 0.8271

mAP@0.5 geral: 0.9788
mAP@0.5:0.95 geral: 0.7772
Velocidade de inferência: 0.9ms


In [1]:
import shutil

shutil.copy('runs/detect/train_phase2/weights/best.pt', 'YoloULTRAmegaBLASTIKtralala2.pt')

'YoloULTRAmegaBLASTIKtralala2.pt'